# Project 2a: Alloy cluster expansions

In this project you will construct an alloy cluster expansion model for the Ag-Pd alloy.
Note that it is not required for you to understand all the details of the cluster-expansion formalism in order to solve the project.
Rather you will be using an external package for computing the fit/sensing matrix and compiling the target vector.

**You should hand in a report, via Canvas, maximum 6 pages (excluding references). 
Do not attach any appendices. In the report you should present your data, models and the final results.
Do not forget to visualize your data and results.
It is important that you discuss your findings and reflect on the results.
In addition you need to hand in your Python code.
The code itself will not be graded, but it should run without errors and upon inspection reproduce the main results you present in the report.
Emphasis will be put on your ability to use appropriate terminology to describe and discuss the employed statistical methodologies and results.**

**<font color=red>DEADLINE: Nov 27, 23:59</font>**


## Alloys

Alloys are multicomponent systems, in which several species are mixed with each other on at least one sublattice.
Examples include metallic alloys such as steel or bronze as well as semiconductor alloys such as gallium indium nitride (blue lasers).
In order to study the energetics of such systems on the atomistic scale one often resorts to density functional theory (DFT) calculations, which can provide the energy of a given atomistic structure with high accuracy.
The mixing energy per atom of a structure is defined as
\begin{equation}
\large
    E_\text{mix} = \frac{E_\text{structure} - n_\text{Ag} E_\text{Ag} - n_\text{Pd} E_\text{Pd}}{n_\text{Ag} + n_\text{Pd}}
\end{equation}
where $E_\text{structure}$ is the total energy of the structure, $n_\text{Ag}$ number of Ag atoms in the structure and $E_\text{Ag}$ the energy of pure Ag.
Working with the mixing energy is often convient as it is the relevant energy when considering e.g. phase stability of alloys.
Below a few different structures (configurations) are displayed.

<img src="images/configurations.png" alt="Drawing" style="width: 750px;"/>



## The configurational space problem
If we consider a binary system (A-B) and a lattice consisting of 100 atoms, this yields $2^{100}\approx 10^{30}$ ways of occupying the lattice.
While many of the structures will be duplicates when considering translation and rotational invariance we are still left with a huge number of configurations that would need to be considered, e.g., when calculating thermodynamic averages or conducting ground state searches.
Calculating the energy of multiple structures with DFT is computationally very expensive and thus to solve this configurational space problem one often turns to building atomistic models based on DFT.
So-called alloy cluster expansions are the most commonly employed technique in this context.


## Cluster expansions
The alloy cluster expansion model is based on an ideal lattice.
This lattice can be occupied by various atomic species (Ag and Pd in our case).
In this approach the energy of the atomic structure is modeled as a sum over the average number of times a particular cluster occurs multiplied with corresponding effective cluster interactions (ECIs).
Formally
\begin{equation}
    \large
    E_\text{mix} = J_0 + \sum _\alpha N_\alpha J_\alpha,
\end{equation}
where $\alpha$ refers to various clusters such as nearest neighbor pairs, next nearest neighbor pairs, triplets etc, $N_\alpha$ refers to the number of such clusters for the present structure per atom and $J_\alpha$ is the corresponding effective cluster interaction (ECI) for cluster $\alpha$.
In this project, it is of no concern how the extraction of identical and unique clusters using crystal symmetries is done.

<img src="images/clusters_v1.png" alt="Drawing" style="width: 650px;"/>

Instead we will focus on how to determine the unknown parameters (ECIs) $J_\alpha$, and how to choose which type of clusters to include in the model, i.e. the size and structure of the parameter space.
If the clusters $\alpha$ are enumerated, the equation above can be formulated as a dot product by defining $\boldsymbol{x}=[1, N_1, N_2, \ldots]$ and $\boldsymbol{J}=[J_0, J_1, J_2, \ldots]$ as $E_\text{mix} = \boldsymbol{x} \boldsymbol{J}$.
Here, $\boldsymbol{x}$ is often refered to as the cluster vector of the given configuration.

If the mixing energy $E_\text{mix}$ is calculated using DFT for a set of configurations, then the problem of finding the ECIs $\boldsymbol{J}$ can be solved, e.g., using ordinary least-squares (OLS)
\begin{equation}
    \large
    \boldsymbol{J}_\text{opt} = \min_\boldsymbol{J} || \boldsymbol{X} \boldsymbol{J} - \boldsymbol{E}_\text{mix} ||_2 ^2,
\end{equation}
where $\boldsymbol{E}_\text{mix}$ is a vector containing the mixing energies for all structures and $\boldsymbol{X}$ a matrix, each row of which corresponds to a cluster vector of one of the structures.


## The linear problem
While OLS is the simplest method for finding ECIs, it has a tendency for overfitting.
Linear regression with regularization such as ridge or LASSO are therefore favorable alternatives.
A common issue in finding the ECIs is to select an approriate number of features (clusters) to include in the model.
Features in our problem correspond to clusters with different interatomic distances and of different order (pairs, triplets etc; see above).
Including too few (many) can lead to underfitting (overfitting).

In this project you will construct cluster expansion models using linear regression methods such as Lasso and ARDR.
You will use cross-validation (CV) but also information criteria such as AIC and BIC in order to evaluate how many features are reasonable to include in your model.
For more information and details about these methods and metrics, consult the section "Additional Information" below.
Lastly, you will see how Bayesian parameter estimation of the ECIs can be used to analyze results obtained via a cluster expansion.



### Cross-validation
You may freely choose a suitable cross-valdation method to use for the following tasks.
K-fold is commonly used and available through `sklearn` (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html).
Ten folds might be a good starting point to try out.


### Information criteria
The Bayesian information criteria (BIC) is defined as
\begin{equation}
\text{BIC} = 2 \log(L_*) - N_p \log(N_d)
\end{equation}
where $L_*$ is the maximized likelihood, $N_p$ the number of parameters, and $N_d$ number of data points.     
The energies (or data) can be modeled through
Here we assume the errors, $\epsilon$, are normal distributed with zero mean and variance $\sigma^2$.
\begin{equation}
\boldsymbol{E} = \boldsymbol{x} \boldsymbol{J} + \epsilon , \quad \epsilon \sim \mathcal{N}(\mu=0, \sigma^2) \\
\end{equation}
Therefore the likelihood, $P(D|\boldsymbol{J}, \sigma)$, can be defined as
\begin{equation}
L = P(D|\boldsymbol{J}, \sigma) = \frac{1}{(2\pi \sigma^2)^{N_d/2}}\exp{(-||\boldsymbol{X} \boldsymbol{J}-\boldsymbol{E} ||^2 / 2\sigma^2)}
\end{equation}
The ECIs, $\boldsymbol{J}$, you will obtain from linear regression algorithms (LASSO, etc), and $\sigma$ can be found from setting $\frac{\mathrm{d} L }{\mathrm{d} \sigma} = 0$. This is left as an exercise to the reader but the final expression for BIC becomes
\begin{equation}
\text{BIC} = -N_d \log(\text{MSE}) - N_p \log(N_d) + \text{const}
\end{equation}
where the constant term, $\text{const}$, is irrelevant and $\text{MSE}$ is the mean squared error, defined as 
\begin{equation}
\text{MSE} = \frac{1}{N_d}||\boldsymbol{X} \boldsymbol{J}-\boldsymbol{E} ||^2
\end{equation}

### sklearn
Regression algorithms such as Lasso, ridge, automatic relevance detection (ARD) and many more are available via [`sklearn`](https://scikit-learn.org/).

*Note: Since we are including the constant term in our cluster vectors (corresponding to $J_0$ and implying that the first element is always 1), we need to set `fit_intercept=False` when using the linear models from sklearn.*

## Additional information
Additional information in regards to cluster expansions and how these can be trained and used can be found in the  following papers:

* [*Robust data-driven approach for predicting the configurational energy of high entropy alloys*](https://doi.org/10.1016/j.matdes.2019.108247) (doi:10.1016/j.matdes.2019.108247)
* [*ICET – A Python Library for Constructing and Sampling Alloy Cluster Expansions*](https://doi.org/10.1002/adts.201900015) (doi:10.1002/adts.201900015)

## Setup

In this project you will need to use the [`ase` package](https://wiki.fysik.dtu.dk/ase) package for reading and handling atomistic structures.
We will also use the [`icet` package](https://icet.materialsmodeling.org/) (integrated cluster expansion toolkit) devloped here at Chalmers in order to construct cluster vectors.
Both of these packages are pre-installed on the jupyterhub instance for the course.

In [ ]:
import ase
import icet

# Task 1: Collect and plot the data (0.5p)
Start by reading and collecting the atomic structures and their corresponding properties (such as mixing energy) from the [ASE database](https://wiki.fysik.dtu.dk/ase/ase/db/db.html) located in `structures/reference_data.db`.

The structures are available as [ASE Atoms](https://wiki.fysik.dtu.dk/ase/ase/atoms.html) objects which have attributes such as
* positions
* cell
* numbers (atomic numbers)
* symbols (atomic symbols)

This database can for example be read by the following code snippet

In [ ]:
from ase.db import connect

db = connect('structures/reference_data.db')
for row in db.select():
    atoms = row.toatoms()
    E_mix = row.mixing_energy
    print(row.symbols, E_mix)

Here each row corresponds to an atomic structure with an attached mixing energy in units of meV per atom.      

**Task:** 
* Read the database and collect the atomic structures and energies.
* Plot the mixing energy as a function of Pd concentration

## Task 2: Cutoff selection for a pair cluster-expansion  (5p)
In this task you will construct cluster expansions based only on pairs.
Your cluster expansion thus has a hyper-parameter, namely the pair cutoff, which needs to be determined.
For this task you will use `icet` to construct cluster vectors.
This can be done by first creating a [`ClusterSpace` object](https://icet.materialsmodeling.org/moduleref_icet/cluster_space.html) which requires a list of cutoffs.

In [ ]:
from icet import ClusterSpace

# setup CS
cutoffs = [6, 5, 4]
prim = db.get(1).toatoms()
cs = ClusterSpace(prim, cutoffs=cutoffs, chemical_symbols=['Ag', 'Pd'])

# get cluster-vector for a given atoms object
atoms = db.get(3).toatoms()
x = cs.get_cluster_vector(atoms)
print(x.shape)

In the code above cutoffs of 6 Å for pairs, 5 Å for triplets and 4 Å for quadruplets are used.
When only using pairs, as in this task, simply use `cutoffs=[r2]`, where `r2` is the pair-wise cutoff.

**Tasks:**
* For a range of pair cutoffs, use OLS to compute the training and CV errors and the information criteria AIC and BIC.
* How would you based on these error/metrics select your pair cutoff, discuss and explain your reasoning.


*Note: Since the constant term is included in our cluster vectors we need to set `fit_intercept=False` when using the linear models from sklearn.*

## Task 3:  Feature selection  (5p)
An alternative approach to "manually" finding good values for the cutoffs is to simply select a long-ranged model and use feature-selection methods to automatically select which cluster interactions (parameters) are important and which are not.

In this task you will try out the feature selection approach by using cutoffs `[13, 8, 6]`, and employing LASSO and ARDR. (Feel free to try out other feature selection algorithms like e.g. OMP, RFE)

LASSO has a hyper-parameter `alpha` which determines the sparsity of the solution (number of nonzero parameters). ARDR has a hyper-parameter `threshold_lambda` which sets how many nonzero parameters appear in the solution.

See the documentation here
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
* https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ARDRegression.html


Scan a range of values for the respective hyper-parameter and evaluate CV-error and IC to determine a good value for the hyper-parameters. Records also how many nonzero parameters you obtained.

**Tasks:**
* Standardize the data. Why is this important for these two feature selection algorithms?
* Compute the training and CV error for a range of values for the hyper-parameters.
* Visualize the training error and CV-error as well as the AIC/BIC as function of number of non-zero parameters in the model.    

**Discuss:**
* How many features you think is suitable to include in a final model based on your analysis.
* Does LASSO and ARDR select the same features?
* Discuss the differences between LASSO and ARDR.

*Note: Since the constant term is included in our cluster vectors we need to set `fit_intercept=False` when using the linear models from sklearn.*

## Task 4: Bayesian Cluster expansion   (7p)
In this final task you will try a bayesian approach to constructing cluster expansions.     
In this task use cutoffs `cutoffs=[12.0, 6.0]`.


### Priors
For this problem you can use a gaussian prior for the ECIs with a zero mean and variance $\alpha ^2$. This can be defined as
\begin{equation}
P(\boldsymbol{J}) = \frac{1}{(2\pi \alpha^2)^{N_p/2}}\exp{(-||\boldsymbol{J}||^2 / 2\alpha^2)}
\end{equation}
where $N_p$ is the number of ECIs (length of vector $\boldsymbol{J}$).
For the priors over $\sigma$ and $\alpha$ you can for example use inverse gamma distributions. The total prior $P(\boldsymbol{J}, \sigma, \alpha)$  is then given by the product of all priors.

### Posterior
In this task you will generate cluster expansion models from the posterior probability distribution $P(\boldsymbol{J}|D)$, where $D$ is the training data (cluster-vectors and energies). The posterior is defined as
\begin{equation}
\underbrace{P(\boldsymbol{J}, \sigma, \alpha|D)}_\text{posterior} = \underbrace{P(D|\boldsymbol{J}, \sigma)}_\text{likelihood} \underbrace{P(\boldsymbol{J}, \sigma, \alpha)}_\text{prior}/P(D)
\end{equation}


**Tasks:**
* Define the (log) liklihood, (log) prior and (log) posterior functions.
* Sample the posterior distribution using MCMC and generate about 100-1000 (uncorrelated) samples (models).
* Visualize the ECIs for a MCMC chain.


### The ground state

The ground-state structure refers to the structure with the lowest energy for a given concentration.
Macroscopic properties like thermal conductivity, heat-capacity etc can depend strongly on the particular ground-state configuration and it is therefore important to know the correct ground-state structure.

In the database `structures/ground_states_candidates.db` you will find potential ground-state candidates for $c_{Pd}=0.667$. In this task you will assign each ground-state candidate a probability of being the actual ground-state.


**Tasks:**
* Use OLS to predict which structure is the ground-state and the corresponding ground-state energy.
* Use the cluster-expansions samples (from MCMC) and compute the frequency of each candidate structure being the ground-state.
* Plot the disitrubtion of the ground-state energy.
* Compare the results from the previous two tasks with the OLS results and discuss.


*Hints:*
* `emcee` can be used for the MCMC sampling
* You may need to use a long burn-in period
* From previous tasks we expect $\sigma$ to be roughly in the range 1-10meV/atom, and $\alpha$ in roughly the range 5-50meV. 